# **ALPHA VOLATILITY GENERATION - SYSTEMATIC TRADING STRATEGIES PROJECT**

In [1]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import importlib

In [2]:
from Data.market_data import Market_data
from backtester import Backtester

In [3]:
from strategy.base_strat import BaseStrategy
from strategy.regression_network import Regression_realvol, Regression_IV, Regression_IVvsRV

In [4]:
df_train = pd.read_pickle("df_train.pkl")
df_validation = pd.read_pickle("df_validation.pkl")
df_test = pd.read_pickle("df_test.pkl")
df_price = pd.read_pickle("df_price.pkl")
df_option = pd.read_pickle("df_merged.pkl")

In [5]:
data = Market_data(df_train,df_validation,df_test,df_price,df_option)

----

----

# **Statistical strategy : Strategy based on heuristic statistical approach to understand volatilities dynamics**

As a first step, we use a manual and heuristic statistical approach to study and predict volatility. Instead of using complex mathematical models or machine learning, we build and test alphas based on simple observations, statistical patterns, and market intuition. The core idea is to leverage well-known features of volatility such as clustering, mean reversion, term structure, the gap between implied and realized volatility, and cross-reactions between IV and RV and translate them into tradable signals.

**Advantages**:
- Allows quick exploration of potential behaviors without complex models.
- Builds an initial understanding and intuition of IV vs RV dynamics.
- Strategies are easy to interpret, reproduce, and explain.

**Drawbacks**:
- Can quickly become obsolete, fragile, and unable to capture complex dynamics.
- Limited generalization power.
- Markets often behave counter-intuitively: simple rules like “if volatility goes up, it will keep going up” can be misleading. The most robust signals are sometimes the ones that seem the most counter-intuitive.

### **1st strategy**:

For a given quoted straddle, we look at the absolute log-returns of the last 3 days and the average IV (all maturities) over the same period. 
The strategy that performed best on df_train is the following:

- **LONG signal:**  
  If |log-return| is increasing and IV is also increasing → **LONG**

- **SHORT signal:**  
  If |log-return| is decreasing and IV is also decreasing →  **SHORT**.

In [6]:
from strategy.statistical_strat import Statistical_strat1

In [7]:
stat_strat1 = Statistical_strat1(data)
back = Backtester(data,stat_strat1)

In [8]:
back.run_backtest_IVvsRV() # IV under RV over and IV over RV under

IV vs RV BACKTEST RESULTS - TRAIN SET
Total trades: 603
LONG trades: 276 - (RV > IV) Success rate: 42.03% versus 38% in df_train
SHORT trades: 327 - (RV < IV) Success rate: 57.19% versus 62% in df_train
OVERALL success rate: 50.25%

IV vs RV BACKTEST RESULTS - VALIDATION SET
Total trades: 125
LONG trades: 62 - (RV > IV) Success rate: 19.35% versus 25% in df_validation
SHORT trades: 63 - (RV < IV) Success rate: 85.71% versus 75% in df_validation
OVERALL success rate: 52.80%


In [9]:
back.run_backtest_train()

Result on df_train : PNL:227.9232588989485, ROI:3.930341570268134 %


In [10]:
back.run_backtest_validation()

Result on df_validation : PNL:-13.14461460209089, ROI:-1.323914207651722 %


**Observation** : The strategy was  performant in the df_train especially to catch LONG alpha. On df_validation backtest, it worked very well for SHORT signals, as shown by the results. However, the LONG signals were too frequent and not strong or selective enough, which weakened the PnL performance generated by the good SHORT signals.

### **2nd strategy**: 
### **term structure statistical analysis**

For this second strategy, we studied the term structure of implied volatility. In `exploratory_analysis2`. Since we were unable to generate a long vol alpha that outperforms the LONG accuracy on df_train straddles (38%) due to the structural bias in option prices, we will focus on short vol signals this way, we will have a higher chance of generating positive PnL.

We then tried to improve the signal: for upward-sloping term structures, we measured the slope using a linear regression. The idea was: if the slope is above a certain quantile, relative to all slopes calculated on df_train, then we short.

However, after calibrating this threshold on df_train, the results were not significantly better. Therefore, we keep the initial, simpler strategy.

- **SHORT signal:**  
  If the term structure is increasing → **SHORT**

In [11]:
from strategy.statistical_strat import Statistical_strat2

In [12]:
stat_strat2 = Statistical_strat2(data)
back = Backtester(data,stat_strat2)

In [13]:
back.run_backtest_IVvsRV() # IV under RV over and IV over RV under

IV vs RV BACKTEST RESULTS - TRAIN SET
Total trades: 872
LONG trades: 0 - (RV > IV) Success rate: 0.00% versus 38% in df_train
SHORT trades: 872 - (RV < IV) Success rate: 66.51% versus 62% in df_train
OVERALL success rate: 66.51%

IV vs RV BACKTEST RESULTS - VALIDATION SET
Total trades: 201
LONG trades: 0 - (RV > IV) Success rate: 0.00% versus 25% in df_validation
SHORT trades: 201 - (RV < IV) Success rate: 52.74% versus 75% in df_validation
OVERALL success rate: 52.74%


In [14]:
back.run_backtest_train()

Result on df_train : PNL:69.48952439484158, ROI:1.067990114527956 %


In [15]:
back.run_backtest_validation()

Result on df_validation : PNL:7.094793381853656, ROI:0.5236280384856531 %


### **3rd strategy**: 
### **strategy based on the spread**

We study a strategy based on the spread : historical_RV(day_to_mat) - IV

We have been able to observe in `exploratory_analysis2` file that when historical volatility is significantly higher than implied volatility (i.e., when the spread RV – IV is above the 90th quantile among positive spreads), the option tends to be overpriced.

Conversely, when the spread RV – IV is below the 20th quantile among negative spreads (i.e., when IV is much larger than historical volatility), then IV tends to be underpriced with an accuracy of 52% which is better than the random chance = 38% of accuracy on df_train because of the structural bias on option vol prices .

Let's retain the following strategy :

- **SHORT signal:**  
  If historical_RV > (IV with spread > quantile(90%)) among positive spread → **SHORT**

- **LONG signal:**  
  If historical_RV < (IV with spread < quantile(10%)) among negative spread → **LONG**

In [16]:
from strategy.statistical_strat import Statistical_strat3

In [17]:
stat_strat3 = Statistical_strat3(data)
back = Backtester(data,stat_strat3)

In [18]:
back.run_backtest_IVvsRV() # IV under RV over and IV over RV under

IV vs RV BACKTEST RESULTS - TRAIN SET
Total trades: 1193
LONG trades: 693 - (RV > IV) Success rate: 52.53% versus 38% in df_train
SHORT trades: 500 - (RV < IV) Success rate: 64.40% versus 62% in df_train
OVERALL success rate: 57.50%

IV vs RV BACKTEST RESULTS - VALIDATION SET
Total trades: 99
LONG trades: 94 - (RV > IV) Success rate: 17.02% versus 25% in df_validation
SHORT trades: 5 - (RV < IV) Success rate: 100.00% versus 75% in df_validation
OVERALL success rate: 21.21%


In [19]:
back.run_backtest_train()

Result on df_train : PNL:988.9404623719468, ROI:6.834934092284402 %


In [20]:
back.run_backtest_validation()

Result on df_validation : PNL:-210.78644624393996, ROI:-23.43165101980257 %


**REMARK**: Although this strategy looked particularly effective on df_train based on the descriptive statistics from `exploratory_analysis2`, which suggested a link between the spread (historical RV – IV) and the actual IV vs RV outcome, it does not seem to generalize well on df_validation. There were not SHORT signal enough to compensate loss PNL due to bad LONG alphas. As shown by the result above, it leads to a -23% loss.


## **Statistical analysis on skew :**

On this part we focus on skew analysis to build alpha

### **4th strategy**:

### **Risk Reversal Delta 25**

This strategy is based on the risk reversal statistic, which compares the implied volatility of the 25-delta put with the 25-delta call.

The chosen strategy is:

- **LONG signal:**  
  If (RR25 – average historical RV > threshold) AND (last 3 IV days are decreasing) → **LONG**

- **SHORT signal:**  
  If (RR25 is decreasing over the last 3 days) AND (RR25 – last RR25 > threshold)  → **SHORT**
  (If RR25 decreases for three consecutive days, but the difference between today’s RR25 and yesterday’s RR25 becomes smaller, then we take a SHORT signal)


PERFORMANCE ON DF_TRAIN : 52.67% accuracy on LONG (>38%) and 71.29% accuracy on SHORT (>62%)

In [21]:
from strategy.statistical_strat import Statistical_strat4

In [22]:
stat_strat4 = Statistical_strat4(data)
back = Backtester(data,stat_strat4)

In [23]:
back.run_backtest_IVvsRV() # IV under RV over and IV over RV under

IV vs RV BACKTEST RESULTS - TRAIN SET
Total trades: 657
LONG trades: 131 - (RV > IV) Success rate: 52.67% versus 38% in df_train
SHORT trades: 526 - (RV < IV) Success rate: 71.29% versus 62% in df_train
OVERALL success rate: 67.58%

IV vs RV BACKTEST RESULTS - VALIDATION SET
Total trades: 129
LONG trades: 31 - (RV > IV) Success rate: 45.16% versus 25% in df_validation
SHORT trades: 98 - (RV < IV) Success rate: 79.59% versus 75% in df_validation
OVERALL success rate: 71.32%


In [24]:
back.run_backtest_train()

Result on df_train : PNL:271.02061749902936, ROI:4.354521328285516 %


In [25]:
back.run_backtest_validation()

Result on df_validation : PNL:54.227469398688385, ROI:5.58205890090054 %


**Observation** : This Risk Reversal strategy outperformed the market and was able to generate a positive PnL on the validation sample (df_validation).

### **5th strategy**:
### **Regression network : Strategy based on a linear regression between implied volatility (IV) and realized volatility (RV)**

The strategy takes advantage of differences between implied volatility (IV) and realized volatility (RV) to find trading opportunities. For each option, the last 10 dates are considered, and a linear regression between IV and RV is performed to extract the slope, which reflects the recent trend. Here is the momentum strategy we have chosen **based on what worked best during the backtesting**  :

- **LONG signal:**  
  If (the IV slope is above the 80th percentile of past slopes) **and** (the RV slope is below the 20th percentile), IV is overestimated compared to RV, and the strategy goes **LONG**.

- **SHORT signal:**  
  Conversely, if (the IV slope is below the 20th percentile) **and** (the RV slope is above the 80th percentile), IV is underestimated compared to RV, and the strategy goes **SHORT**.

The strategy exploits recent imbalances between implied and realized volatility, buying when the market overestimates volatility and selling when it underestimates it.


**DRAWBACK** : The main drawback of this strategy is that IV is aggregated using a simple daily average (by quote_date of the straddle). As a result, the strategy does not capture the term structure of volatility. The slope obtained from the regression over the last nb_period days only reflects the average IV trend, which is then compared to realized volatility (standard deviation of returns over the last nb_period days), calculated in the same way.

To sum up, the strategy does not take into account the term structure of straddles during the backtest. Moreover, the choice of quantiles was arbitrary, based on exploratory data analysis, and the window size (nb_period = 10) was also chosen in the same way.

In [6]:
regrv = Regression_realvol(data, 10)
regiv = Regression_IV(data, 10)
regivrv = Regression_IVvsRV(regiv, regrv, data, 10)

In [7]:
back = Backtester(data,regivrv)

In [8]:
back.run_backtest_IVvsRV() # IV under RV over and IV over RV under

IV vs RV BACKTEST RESULTS - TRAIN SET
Total trades: 850
LONG trades: 202 - (RV > IV) Success rate: 43.56% versus 38% in df_train
SHORT trades: 648 - (RV < IV) Success rate: 64.51% versus 62% in df_train
OVERALL success rate: 59.53%

IV vs RV BACKTEST RESULTS - VALIDATION SET
Total trades: 14
LONG trades: 5 - (RV > IV) Success rate: 100.00% versus 25% in df_validation
SHORT trades: 9 - (RV < IV) Success rate: 100.00% versus 75% in df_validation
OVERALL success rate: 100.00%


In [29]:
back.run_backtest_train()

Result on df_train : PNL:127.29262945552922, ROI:1.5535199632836612 %


In [30]:
back.run_backtest_validation()

Result on df_validation : PNL:26.74093148660205, ROI:23.681306665428668 %


**Observation** : The model really well performed but over only 14 position, the strategy is much too conservative. 

------

-----

# **Garch strat**

This strategy uses a GARCH(1,1) model to predict the realized volatility of the underlying.
I trade only when the predicted volatility is either higher than all implied volatilities (IVs) or lower than all IVs.

- **LONG signal:**  
  If predicted vol < all IVs and the option IV is in the lowest quartile → **LONG**.

- **SHORT signal:**  
  If predicted vol > all IVs and the option IV is in the highest quartile → **SHORT**.

The goal is to exploit extreme situations where model forecasts strongly disagree with market IVs. The strategy is calibrated on a rolling window of spot prices using a GARCH model.
It was then adapted and tuned to perform on df_validation, after training the GARCH on sliding spot data.

**Remark and observation** : We notice that as the filter criteria applied to the straddles become stricter, the success rate gradually decreases (48% → 46% → 45% → 41% of correct signals, see ***exploratory_analysis.ipynb***), while the number of trades in the filtered universe remains reasonable. The results thus become inconsistent with the strategy’s intuition. It is therefore better to use the **inverse** signal of the strategy.

In [9]:
from strategy.garch_strat import Garch_strat

In [10]:
stratgarch = Garch_strat(data)
back = Backtester(data,stratgarch)

In [11]:
back.run_backtest_validation()

Result on df_validation : PNL:433.83298361478785, ROI:14.611352865796196 %


The strategy has well performed on df_validation because it was tuned on it

---

---

# **Regime Switching strategy : statistical strat (model free) and Markov regime switching (HMM)**

The idea of the strategies are to exploit sudden regime shifts to build a signal that identifies whether an option is underpriced or overpriced, based on its phase shift relative to the realized volatility. 

An asset’s volatility (whether realized volatility or implied volatility) does not evolve uniformly. It alternates between different “states” or “regimes” (e.g., High Vol / Low Vol), each with its own statistical characteristics.

Retained Strategy : exploit these moments of sudden phase shifts to create a signal:


- If RV suddenly rises but IV lags → the options market underestimates the volatility regime: **LONG** volatility opportunity.

- If RV suddenly drops → one could think the straddles are probably overpriced: **SHORT** volatility opportunity.

### **First Strategy: Regime identification using thresholds (Free-Model and Non-Parametric Approach)**

The first strategy we designed to model volatility regime shifts is a purely statistical, non-parametric approach. 
The idea is to identify volatility regimes using thresholds rather than relying on parametric models.  

For each option, we compute the daily **log-return** and compare it with the **10-day historical volatility**.  
The trading signal is then constructed as follows:  

- **High-volatility regime (LONG signal):**  
  If $|\text{log return}| > \text{threshold}_{high} \times \text{vol}_{10d}$,  
  the market signals a regime shift into a high-volatility state → **LONG straddle**.  

- **Low-volatility regime (SHORT signal):**  
  If $|\text{log return}| < \text{threshold}_{low} \times \text{vol}_{10d}$,  
  the market signals a regime shift into a low-volatility state → **SHORT straddle**.  

The thresholds ($\text{threshold}_{high}, \text{threshold}_{low}$) are calibrated in the 
`exploratory_analysis.ipynb` notebook using the training dataset **df_train**.


In [ ]:
from strategy.regime_switching_strat import Regime_switching_modelfree

In [32]:
reg_switch_freemod = Regime_switching_modelfree(data)
back = Backtester(data,reg_switch_freemod)

d:\Users\Jason\Desktop\ALPHA projet\strategy\regime_switching_strat.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_train_sub = df_train_sub[self.market_data.df_train['Date'] > " 2016-03-01"].copy()


In [33]:
back.run_backtest_IVvsRV()

IV vs RV BACKTEST RESULTS - TRAIN SET
Total trades: 1209
LONG trades: 117 - (RV > IV) Success rate: 60.68% versus 38% in df_train
SHORT trades: 1092 - (RV < IV) Success rate: 63.74% versus 62% in df_train
OVERALL success rate: 63.44%

IV vs RV BACKTEST RESULTS - VALIDATION SET
Total trades: 232
LONG trades: 5 - (RV > IV) Success rate: 40.00% versus 25% in df_validation
SHORT trades: 227 - (RV < IV) Success rate: 62.11% versus 75% in df_validation
OVERALL success rate: 61.64%


In [34]:
back.run_backtest_train()

Result on df_train : PNL:-290.3683528327171, ROI:-2.6202840830417102 %


In [35]:
back.run_backtest_validation()

Result on df_validation : PNL:122.14688399753324, ROI:7.107306718658298 %


**Observation** : The model performed well on the long trades, but it only executed 5 trades, making the strategy too conservative. However, during the validation phase, its accuracy was lower than the one obtained a posteriori. However the PNL remain positif

### **Second Strategy: Regime identification using gaussian HMM**  

In this second part on regime-switching strategies, we apply a Hidden Markov Model (HMM) to capture volatility regimes and their transitions. The model is built with 3 states: low, middle, and high, using the hmmlearn package.

After fitting the model on df_train, it outputs an n × 3 probability matrix, where each row (each date) gives the probabilities of being in each regime. This matrix is row-stochastic. To build a trading signal, we focus only on state 1 (low regime) and state 3 (high regime), with conditions: proba_low > seuil_low and proba_high > seuil_high.

From the threshold calibration (see `exploratory_analysis.ipynb`), the optimized thresholds delivered 52.17% accuracy for LONG signals (vs 38% baseline) and 68.7% for SHORT signals (vs 62% baseline). These results look very promising. However, since the model involves complex parameter optimization, running the gamma scalping simulation is very time-consuming.


The final strategy is :
- **Low-volatility regime (SHORT signal):**  
  If 5-day realized volatility (RV_5d) < IV and the probability of being in the low regime > threshold_low_regime = 0.86 : SHORT  
- **Low-volatility regime (LONG signal):**  
  If 5-day realized volatility (RV_5d) > IV and the probability of being in the low regime > threshold_high_regime = 0.7 : LONG 

RESULT ON df_validation: 

the model achieve However 80.79% accuracy on SHORT signals (vs 75% baseline) among 288 trades. On the other hand, no LONG trades were triggered, which suggests that the applied filter was too restrictive.

The model was then retrained on the entire df_validation dataset, using all available prices up to that period (a posteriori knowledge).
This explains the outstanding results obtained in this setup.

**Converserly** : the following backtest program simulates more realistic market conditions: it trains the model on a rolling basis, gradually incorporating prices as they become available over time.

In [6]:
from strategy.regime_switching_strat import Regime_switching_HMM

In [7]:
reg_switch_hmm = Regime_switching_HMM(data)
back = Backtester(data,reg_switch_hmm)

In [ ]:
# back.run_backtest_IVvsRV() # too long

In [8]:
back.run_backtest_validation()

Result on df_validation : PNL:-86.72638901443008, ROI:-5.923326777613642 %


Unfortunately, after backtesting in real market conditions (with rolling basis training), the model failed to properly capture the volatility dynamics, it has generated -5.9% of ROI.